In [1]:
import pandas as pd
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
# from util import UnitConversion, mapping_list_values, perl_to_posix
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
pd.options.display.max_columns = None
pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
import ast
import warnings
import regex as re
warnings.filterwarnings('ignore')
from decimal import *
TWOPLACES = Decimal(10) ** -2
from natsort import natsorted
import ast
import time
today = time.strftime("%Y_%m_%d")
from enrich_dimensions.rounds import rounds, rounding_inch_feet,rounding_lbs,rounding_w,rounding_oz, rounding_lb,rounding_gal, re_extract, curate, round_string_float, clean_data,reg_ex,rounding_period_after_unit 
from enrich_dimensions.params import parameters, query_from_file
from enrich_dimensions.query_file import query_from_file 
from enrich_dimensions.custom import custom_field 


def get_unique_list(material_list):
    unique_list = []
    for item in material_list:
        if isinstance(item, str) and "[" in item and "]" in item:
            item_values = ast.literal_eval(item)
            unique_list.extend(item_values)
        else:
            unique_list.append(item)
    return sorted(list(set(unique_list)))



customer_id = '11'
customer_name='%crateandbarrel%'
client='Crate and Barrel'

In [2]:
dateszs='2001-08-11'
attribut='features'
dateszs='2001-11-01'
curation_col = f'Q:{attribut}'
params = {'customer_id': customer_id,
          'customer_name':customer_name,
         'dateszs':dateszs,
         'attribut':attribut}

print('Start Query of Curated Data')
cb = query_from_file(file_name='query/curated_all_attributes_date_family.sql', params=params)
print(len(cb))

fill_dict = cb.groupby('external_id')['buckets'].last().to_dict()
cb['buckets'] = cb['buckets'].fillna(cb['external_id'].map(fill_dict))

Start Query of Curated Data
521037


# Check for Attribute bucket Values

In [14]:
print(len(sorted(cb['attribute'].explode().value_counts().reset_index()['index'].to_list())))
lst_attribute=sorted(cb['attribute'].explode().value_counts().reset_index()['index'].to_list())
print('Number of attributes: '+str(len(lst_attribute)))
print('')

# lst_attribute=['collection']
for i in range(len(lst_attribute)):
    attri=lst_attribute[i]
    dateszs='2001-08-11'
    attribut=attri
    dateszs='2001-11-01'
    curation_col = f'Q:{attribut}'
    params = {'customer_id': customer_id,
              'customer_name':customer_name,
             'dateszs':dateszs,
             'attribut':attribut}

    print('Start Buckets')
    print(str(i)+' Attribute: '+str(attri))
    bucket_value = query_from_file(file_name='query/Bucket_Value_Strategy.sql', params=params)

    brand=cb[(cb['value'].astype(str)!='["n/a"]')&(cb['value'].astype(str)!='n/a')&(cb['attribute'].astype(str)==attri)]
    print('Number of SKUs for the attribute '+str(attri)+': '+str(len(brand)))
    list_of_all_buckets=sorted(list(set(bucket_value['buckets'].to_list())))

    brand=cb[(cb['value'].astype(str)!='["n/a"]')&(cb['value'].astype(str)!='n/a')&(cb['attribute'].astype(str)==attri)]
#     print('Number of SKUs for the attribute '+str(attri)+': '+str(len(brand)))
    list_of_all_buckets=sorted(list(set(bucket_value['buckets'].to_list())))

    filtered_dfs = {}
    b_lst=[]

    for i in range(len(list_of_all_buckets)):

        # values that are supposed to be in the buckets
        action_bucket_values=bucket_value[bucket_value['buckets'].astype(str)==list_of_all_buckets[i]]['values'].to_list()

        # values that are actually in the curation
        action=brand[brand['buckets'].astype(str)==list_of_all_buckets[i]]['value'].to_list()

        unique_values=get_unique_list(action)

        target=[x for x in unique_values if x not in action_bucket_values]
        if len(target) > 0:
            target = target[0]
            pat=f'''(?i)({target})|()'''
            bucket_filtered_df=brand[brand['buckets'].astype(str)==list_of_all_buckets[i]]
            bucket_filtered_df['match']=bucket_filtered_df['value'].apply(lambda x: re_extract(pat,str(x)))
            fil=bucket_filtered_df[bucket_filtered_df['match'].astype(str)!='[]']
            filtered_dfs[list_of_all_buckets[i]] = fil
            b_lst.append(list_of_all_buckets[i])
        else:
            pass


    b_lst[0:10]
    valuez = {}
    external_id_list=[]
    for i in range(len(b_lst)):
        valuez[b_lst[i]]=filtered_dfs[b_lst[i]][['external_id','value']]
        external_id_list.append(filtered_dfs[b_lst[i]]['external_id'].tolist())
    import itertools
    flattened_list = list(itertools.chain.from_iterable(external_id_list))
    if len(flattened_list)>0:
        print('Buckets Effected: '+str(len(b_lst)))
        df_name = f'match_{attri}'
        data = {'external_id': flattened_list, f'Q:{attri}': '[]'}
        df_dict = {df_name: pd.DataFrame(data)}
        new_df = df_dict[df_name]
        print('Number of SKUs to be wiped: '+str(len(new_df)))
        print('')
        print('')

        def get_df_name(df):
            name =[x for x in globals() if globals()[x] is df][0]
            return name
        def looks_good(customer, matches,attri): 
            drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/Ready For Upload' 
            matches.to_csv(f'{drive_path}/{client} --{get_df_name(matches)}match_{attri}-{today}.csv',index=False) 
        looks_good(client, new_df,attri) 
    else:
        pass

114
Number of attributes: 114

Start Buckets
0 Attribute: accent_chair_type
Number of SKUs for the attribute accent_chair_type: 1365
Start Buckets
1 Attribute: age_group
Number of SKUs for the attribute age_group: 503
Start Buckets
2 Attribute: apparel_size
Number of SKUs for the attribute apparel_size: 15
Start Buckets
3 Attribute: appliance_cubic_feet
Number of SKUs for the attribute appliance_cubic_feet: 3
Start Buckets
4 Attribute: arm_style
Number of SKUs for the attribute arm_style: 657
Start Buckets
5 Attribute: artist
Number of SKUs for the attribute artist: 97
Start Buckets
6 Attribute: artist_signature
Number of SKUs for the attribute artist_signature: 5
Start Buckets
7 Attribute: bag_size
Number of SKUs for the attribute bag_size: 29
Start Buckets
8 Attribute: base_color
Number of SKUs for the attribute base_color: 578
Buckets Effected: 1
Number of SKUs to be wiped: 1


Start Buckets
9 Attribute: base_material
Number of SKUs for the attribute base_material: 850
Start Buckets

In [13]:
# cb[cb['external_id'].isin(flattened_list)]['value'].explode().value_counts()

# Check Freetext Attributes

In [4]:
patterns = {
    'appliance_cubic_feet': r'(cu ft)|(n\/a)|()',
    'capacity_by_ounce': r'(\d oz)|(n\/a)|()',
    'capacity_by_quart': r'(\d qt)|(n\/a)|()',
    'depth':r'(\d in)|(\d ft)|(n\/a)',
    'diameter':r'(\d in)|(\d ft)|(n\/a)',
    'height':r'(\d in)|(\d ft)|(n\/a)',
    'length':r'(\d in)|(\d ft)|(n\/a)',
    'seat_depth_numerical':r'(\d in)|(\d ft)|(n\/a)',
    'seat_height_numerical':r'(\d in)|(\d ft)|(n\/a)',
    'table_linen_length':r'(\d in)|(n\/a)',
    'table_linen_width':r'(\d in)|(n\/a)'
}

In [5]:
def filter_empty_matches(dataframe, attribute, pattern):
    matches = dataframe[dataframe['attribute'].astype(str) == attribute]['value'].apply(lambda x: re_extract(pattern,str(x)))
    empty_matches = matches.apply(lambda x: len(x) == 0)
    return dataframe[dataframe['attribute'].astype(str) == attribute][empty_matches]

def count_empty_matches(dataframe, attribute, pattern):
    print(f'Number of SKUs for {attribute}: '+str(len(dataframe[dataframe['attribute'].astype(str) == attribute])))
    matches = dataframe[dataframe['attribute'].astype(str) == attribute]['value'].apply(lambda x: re_extract(pattern,str(x)))
    return sum(len(match) == 0 for match in matches)


filtered_dataframes = {}
for attribute, pattern in patterns.items():
    filtered_dataframes[attribute] = filter_empty_matches(cb, attribute, pattern)
    
counts = {}
for attribute, pattern in patterns.items():
    counts[attribute] = count_empty_matches(cb, attribute, pattern)
#     print(f'Rows that need to be wiped for {attribute}: '+f': {counts[attribute]}')
   

    try:
        flattened_list = filtered_dataframes[attribute]['external_id'].to_list()
        if len(flattened_list) > 0:
            df_name = f'match_{attribute}'
            data = {'external_id': flattened_list, f'Q:{attribute}': '[]'}
            df_dict = {df_name: pd.DataFrame(data)}
            new_df = df_dict[df_name]
            print('Number of SKUs to be wiped: ' + str(len(new_df)))
            print('')
            print('')

            def get_df_name(df):
                name = [x for x in globals() if globals()[x] is df][0]
                return name

            def looks_good(customer, matches, attribute):
                today = pd.Timestamp.today().strftime("%Y-%m-%d")
                drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/Ready For Upload'
                matches.to_csv(f'{drive_path}/{client} --{get_df_name(matches)}match_{attribute}-{today}.csv', index=False)
            looks_good(client, new_df, attribute)
            
        else:
            print('No SKUs to be wiped for ' + attribute)
    except KeyError:
        print('Error: DataFrame with name ' + df_name + ' does not exist')

Number of SKUs for appliance_cubic_feet: 9
No SKUs to be wiped for appliance_cubic_feet
Number of SKUs for capacity_by_ounce: 2427
No SKUs to be wiped for capacity_by_ounce
Number of SKUs for capacity_by_quart: 1485
Number of SKUs to be wiped: 7


Number of SKUs for depth: 7009
Number of SKUs to be wiped: 2


Number of SKUs for diameter: 2897
No SKUs to be wiped for diameter
Number of SKUs for height: 4616
Number of SKUs to be wiped: 2


Number of SKUs for length: 3
No SKUs to be wiped for length
Number of SKUs for seat_depth_numerical: 613
No SKUs to be wiped for seat_depth_numerical
Number of SKUs for seat_height_numerical: 1352
No SKUs to be wiped for seat_height_numerical
Number of SKUs for table_linen_length: 66
No SKUs to be wiped for table_linen_length
Number of SKUs for table_linen_width: 33
No SKUs to be wiped for table_linen_width


In [ ]:
# width
# depth
# height
# capacity_by_ounce
# diameter
# seat_depth_numerical

In [712]:
print(len(sorted(cb['attribute'].explode().value_counts().reset_index()['index'].to_list())))
sorted(cb['attribute'].explode().value_counts().reset_index()['index'].to_list())[105:]

111


['type_crate_sunbrella',
 'umbrella_size',
 'vase_type',
 'width',
 'wood_finish',
 'wood_type']

In [739]:
attri='wood_type'
dateszs='2001-08-11'
attribut=attri
dateszs='2001-11-01'
curation_col = f'Q:{attribut}'
params = {'customer_id': customer_id,
          'customer_name':customer_name,
         'dateszs':dateszs,
         'attribut':attribut}

print('Start Buckets')
bucket_value = query_from_file(file_name='query/Bucket_Value_Strategy.sql', params=params)

Start Buckets


In [740]:
brand=cb[(cb['value'].astype(str)!='["n/a"]')&(cb['value'].astype(str)!='n/a')&(cb['attribute'].astype(str)==attri)]
print(len(brand))
list_of_all_buckets=sorted(list(set(bucket_value['buckets'].to_list())))

72


# Curated

In [741]:
filtered_dfs = {}
b_lst=[]

def get_unique_list(material_list):
    unique_list = []
    for item in material_list:
        if isinstance(item, str) and "[" in item and "]" in item:
            item_values = ast.literal_eval(item)
            unique_list.extend(item_values)
        else:
            unique_list.append(item)
    return sorted(list(set(unique_list)))


for i in range(len(list_of_all_buckets)):
    
    # values that are supposed to be in the buckets
    action_bucket_values=bucket_value[bucket_value['buckets'].astype(str)==list_of_all_buckets[i]]['values'].to_list()
    
    # values that are actually in the curation
    action=brand[brand['buckets'].astype(str)==list_of_all_buckets[i]]['value'].to_list()
    
    unique_values=get_unique_list(action)
    
#     print(action_bucket_values)
#     print(action)
#     print(brand[brand['buckets'].astype(str)==list_of_all_buckets[i]]['buckets'])
#     print(unique_values)
#     print('')
    
    

        
    target=[x for x in unique_values if x not in action_bucket_values]
#     print(list_of_all_buckets[i])
#     print(target)
#     print('')
    if len(target) > 0:
        target = target[0]
        pat=f'''(?i)({target})|()'''
        bucket_filtered_df=brand[brand['buckets'].astype(str)==list_of_all_buckets[i]]
        bucket_filtered_df['match']=bucket_filtered_df['value'].apply(lambda x: re_extract(pat,str(x)))
        fil=bucket_filtered_df[bucket_filtered_df['match'].astype(str)!='[]']
        filtered_dfs[list_of_all_buckets[i]] = fil
        b_lst.append(list_of_all_buckets[i])
    else:
        pass

In [742]:
print(len(b_lst))
b_lst[0:10]
valuez = {}
external_id_list=[]
for i in range(len(b_lst)):
    valuez[b_lst[i]]=filtered_dfs[b_lst[i]][['external_id','value']]
    external_id_list.append(filtered_dfs[b_lst[i]]['external_id'].tolist())
import itertools
flattened_list = list(itertools.chain.from_iterable(external_id_list))

0


In [743]:
# print(len(brand[brand['external_id'].isin(flattened_list)]))
brand[brand['external_id'].isin(flattened_list)]['buckets'].explode().value_counts()

Series([], Name: buckets, dtype: int64)

In [744]:
brand[brand['external_id'].isin(flattened_list)]['value'].explode().value_counts()

Series([], Name: value, dtype: int64)

In [745]:
# brand[brand['external_id'].isin(flattened_list)]

In [597]:
err

NameError: name 'err' is not defined

In [653]:
print(attri)
print(attri)
print(attri)

stroller_type
stroller_type
stroller_type


In [654]:
match_stroller_type = pd.DataFrame({'external_id': flattened_list, f'Q:{attri}': '[]'})
print(len(match_stroller_type))

2


In [655]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
def looks_good(customer, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/Ready For Upload' 
    matches.to_csv(f'{drive_path}/{client} --{get_df_name(matches)}match_stroller_type-{today}.csv',index=False) 
looks_good(client, match_stroller_type) 

In [751]:
# cb[(cb['attribute'].astype(str)=='appliance_cubic_feet')&(cb['buckets'].astype(str)=='Microwaves')]['value'].explode().value_counts()

In [754]:
# cb[(cb['attribute'].astype(str)=='brand')&(cb['buckets'].astype(str)=='Baby Jumpers, Swings, & Rockers')]#['value'].explode().value_counts()

In [755]:
# cb[(cb['attribute'].astype(str)=='brand')&(cb['buckets'].astype(str)=='Baby Travel Seats')]['value'].explode().value_counts()

Nuna          12
Clek           4
n/a            3
Nuna revv™     1
Name: value, dtype: int64

In [758]:
cb[(cb['attribute'].astype(str)=='capacity_by_ounce')]['value'].explode().value_counts()[0:10]

n/a       2184
20 oz       13
16 oz       11
9 oz        11
22 oz       10
27 oz       10
12 oz        8
6.5 oz       8
10 oz        7
6 oz         7
Name: value, dtype: int64

In [763]:
x=cb[(cb['attribute'].astype(str)=='capacity_by_ounce')]#['value'].explode().value_counts()
pat='''(?i)(n\/a)|(\d+(?:\.\d+)?\soz)|()'''
x['match']=x['value'].apply(lambda x: re_extract(pat,str(x)))
y=x[x['match'].astype(str)=='[]']
print(len(y))

0


In [767]:
# cb[(cb['attribute'].astype(str)=='collection')]#['buckets'].explode().value_counts()

In [769]:
x=cb[(cb['attribute'].astype(str)=='depth')]#['value'].explode().value_counts()

pat='''(?i)(n\/a)|(\d+(?:\.\d+)?\sin)|()'''
x['match']=x['value'].apply(lambda x: re_extract(pat,str(x)))
y=x[x['match'].astype(str)=='[]']
print(len(y))
y['Q:depth']='[]'
match_depth=y[['external_id','Q:depth']]

1155


In [794]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
def looks_good(customer, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/Ready For Upload' 
    matches.to_csv(f'{drive_path}/{client} --{get_df_name(matches)}match_seat_height_numerical-{today}.csv',index=False) 
looks_good(client, match_seat_height_numerical) 

In [780]:
x=cb[(cb['attribute'].astype(str)=='diameter')]#['value'].explode().value_counts()

pat='''(?i)(n\/a)|(\d+(?:\.\d+)?\sin)|()'''
x['match']=x['value'].apply(lambda x: re_extract(pat,str(x)))
y=x[x['match'].astype(str)=='[]']
print(len(y))
y['Q:diameter']='[]'
match_diameter=y[['external_id','Q:diameter']]

49


In [785]:
x=cb[(cb['attribute'].astype(str)=='height')]#['value'].explode().value_counts()

pat='''(?i)(n\/a)|(\d+(?:\.\d+)?\sin)|()'''
x['match']=x['value'].apply(lambda x: re_extract(pat,str(x)))
y=x[x['match'].astype(str)=='[]']
print(len(y))
y['Q:height']='[]'
match_height=y[['external_id','Q:height']]

158


In [787]:
cb[(cb['attribute'].astype(str)=='length')]['value'].explode().value_counts()

n/a    3
Name: value, dtype: int64

In [790]:
x=cb[(cb['attribute'].astype(str)=='seat_depth_numerical')]#['value'].explode().value_counts()

pat='''(?i)(n\/a)|(\d+(?:\.\d+)?\sin)|()'''
x['match']=x['value'].apply(lambda x: re_extract(pat,str(x)))
y=x[x['match'].astype(str)=='[]']
print(len(y))
# y['Q:height']='[]'
# match_height=y[['external_id','Q:height']]

0


In [793]:
x=cb[(cb['attribute'].astype(str)=='seat_height_numerical')]#['value'].explode().value_counts()

pat='''(?i)(n\/a)|(\d+(?:\.\d+)?\sin)|()'''
x['match']=x['value'].apply(lambda x: re_extract(pat,str(x)))
y=x[x['match'].astype(str)=='[]']
print(len(y))
y['Q:seat_height_numerical']='[]'
match_seat_height_numerical=y[['external_id','Q:seat_height_numerical']]

2


In [796]:
x=cb[(cb['attribute'].astype(str)=='table_linen_length')]#['value'].explode().value_counts()

pat='''(?i)(n\/a)|(\d+(?:\.\d+)?\sin)|()'''
x['match']=x['value'].apply(lambda x: re_extract(pat,str(x)))
y=x[x['match'].astype(str)=='[]']
print(len(y))
# y['Q:seat_height_numerical']='[]'
# match_seat_height_numerical=y[['external_id','Q:seat_height_numerical']]

0


In [798]:
x=cb[(cb['attribute'].astype(str)=='width')]#['value'].explode().value_counts()

pat='''(?i)(n\/a)|(\d+(?:\.\d+)?\sin)|()'''
x['match']=x['value'].apply(lambda x: re_extract(pat,str(x)))
y=x[x['match'].astype(str)=='[]']
print(len(y))
# y['Q:seat_height_numerical']='[]'
# match_seat_height_numerical=y[['external_id','Q:seat_height_numerical']]

0


In [27]:
# cb['attribute'].explode().value_counts()

In [43]:
pat='''(?i)(.{0,30}(?:green.?gold).{0,30})|()'''
dfs['matches']=dfs['custom_fields'].apply(lambda x: re_extract(pat,str(x)))
dfs['match']=dfs['long_desc'].apply(lambda x: re_extract(pat,str(x)))

In [44]:
# bif=dfs[(dfs['value'].astype(str)=='n/a')&((dfs['match'].astype(str)!='[]')|(dfs['matches'].astype(str)!='[]'))]
bif=dfs[((dfs['match'].astype(str)!='[]')|(dfs['matches'].astype(str)!='[]'))]
bif=bif[(bif['value'].astype(str)!='Greenguard Gold Certified')&(bif['value'].astype(str)!='["Greenguard Gold Certified","Organic"]')&(bif['value'].astype(str)!='["FSC® Certified","Greenguard Gold Certified"]')]
print(len(bif))

0


In [45]:
bif['value'].explode().value_counts()

Series([], Name: value, dtype: int64)

In [46]:
bif['match'].explode().value_counts()[0:500]

Series([], Name: match, dtype: int64)

In [14]:
fsc=bif[bif['value'].astype(str)=='FSC® Certified']
fsc['Q:sustainability']='["BIFMA Certified","FSC® Certified"]'

In [15]:
na=bif[bif['value'].astype(str)=='n/a']
na['Q:sustainability']='["BIFMA Certified"]'

In [17]:
match__sustainability_na=na[['external_id','Q:sustainability']]
match_sustainability_fsc=fsc[['external_id','Q:sustainability']]

In [29]:
pat='''(?i)(.{0,30}(?:wide|width|\d+.?".?W).{0,30})|()'''
dfs['matches']=dfs['custom_fields'].apply(lambda x: re_extract(pat,str(x)))
dfs['match']=dfs['long_desc'].apply(lambda x: re_extract(pat,str(x)))

In [31]:
print(len(dfs[(dfs['matches'].astype(str)!='[]')|(dfs['match'].astype(str)!='[]')]))

23228


# table_linen_width

In [4]:
df['match']=df['value'].apply(lambda x: re.sub(r'(?i)(50"\s?(?:round|square))','50 x 50',str(x))).apply(lambda x: re.sub(r'(?i)(90"\s?(?:round|square))','90 x 90',str(x))).apply(lambda x: re.sub(r'(?i)(60"\s?(?:round|square))','60 x 60',str(x)))
width=df.loc[df['value'].astype(str)!='n/a']
width['match'].explode().value_counts()

90" x 14"     146
60" x 90"      45
60" x 120"     45
120" x 14"     36
50 x 50        10
60" x 144"      9
90 x 90         6
60 x 60         5
144" x 14"      2
120" x 20"      1
90" x 20"       1
Name: match, dtype: int64

In [7]:
a=[]
for i in range(len(width)):
    x=width['match'][i]#.values
    one,two=str(x).split(' x ')
    one=re.findall(r'\d+', one)
    two=re.findall(r'\d+', two)
    three=one+two
    three=[int(x) for x in three]
#     print(three)
#     print(max(three))
#     print(min(three))
#     print('')
    a.append(min(three))
    b=[str(x)+' in' for x in a]

In [8]:
width['Q:table_linen_width']=b

table_linen_width=width[['external_id','Q:table_linen_width']]
# table_linen_width

In [9]:
width_na=df.loc[df['value'].astype(str)=='n/a']
print(len(width_na))
width_na['Q:table_linen_width']='n/a'
table_width_na=width_na[['external_id','Q:table_linen_width']]

60


In [10]:
lst=width['external_id'].to_list() + width_na['external_id'].to_list()
print(len(lst))

366


In [11]:
df.loc[~df['external_id'].isin(lst)]

,attribute,buckets,bucket_id,value,customer_name,external_id,product_name,long_desc,curated_date,resolution,curation_tasks.curated_by,sku,link,class,title,height,outlet,is_kids,sequence,subgroup,alt_image,furniture,department,dimensions,price_type,description,header_name,large_image,price_range,small_image,is_drop_ship,is_exclusive,medium_image,sku_property,furniture_type,held_in_region,description_full,description_long,is_internet_only,primaryfamily_id,online_avail_code,primary_family_id,is_outlet_purchase,can_be_personalized,current_price_range,is_vendor_drop_ship,on_display_in_store,primary_category_id,regular_price_range,can_be_registered_for,current_price_message,regular_price_message,free_shipping_eligible,attribute_item_group_id,held_in_one_ecomm_location,diameter,average_rating,number_of_ratings,catalog_product_ids,new_product,crate_style,silhouette_image_untrimmed,upc,match


In [109]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
def looks_good(customer, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/Ready For Upload' 
    matches.to_csv(f'{drive_path}/{client} --{get_df_name(matches)}match_out_of_scope_brand-{today}.csv',index=False) 
looks_good(client, match_out_of_scope_brand) 

def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
def looks_good(customer, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/Ready For Upload' 
    matches.to_csv(f'{drive_path}/{client} --{get_df_name(matches)}table_linen_width-{today}.csv',index=False) 
looks_good(client, table_linen_width) 

In [3]:
color=cb[cb['attribute'].astype(str)=='color']

In [6]:
pat='''(?i)(.{0,20}Le.?Creuset.{0,20})|()'''
color['m_name']=color['name'].apply(lambda x: re_extract(pat,str(x)))
color['m_long']=color['long_desc'].apply(lambda x: re_extract(pat,str(x)))
color['m_custom']=color['custom_fields'].apply(lambda x: re_extract(pat,str(x)))

In [ ]:
c=color[(color['m_name'].astype(str)!='[]')|(color['m_long'].astype(str)!='[]')|(color['m_custom'].astype(str)!='[]')]
print(len(c))

# Uncurated

In [ ]:
# parameters
formatted_attribute = 'sustainability'
buckets = """Office Chairs"""

strategy_version_input=767
attribute_id_input=2730

attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
value='%n/a%'
params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'


df = query_from_file(file_name='./query/custom_fields.sql', params=params)
print(len(df))
print(today)

In [113]:
# df['external_id'].to_list()
BIFMA|FSC.{0,5}certified|green.?guard.?gold|organic|standard.?100|OEKO.?TeX

In [2]:
pat='''(?i)(.{0,40}(?i)(?:organic).{0,40})|()'''
df['match']=df['custom_fields'].apply(lambda x: re_extract(pat, str(x)))
sustain=df.loc[df['match'].astype(str)!='[]'][['buckets','external_id','product_name','long_desc','match']]
print(len(sustain))

216


In [4]:
# sustain[0:500]
#['match'].explode().value_counts()[0:500]

In [15]:
stop

NameError: name 'stop' is not defined

# send to the folder for upload

In [16]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute,matches,today): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/BBBY - {attribute}-{get_df_name(matches)}-{today}-matches.csv',index=False) 
    
looks_good('Bed Bath & Beyond', attribute,matchesna,today)   

In [17]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute,matches,today): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/BBBY - {attribute}-{get_df_name(matches)}-{today}-matches.csv',index=False) 
    
looks_good('Bed Bath & Beyond', attribute,matchesdiam,today)  